# Falcon 7B Instruct Model using Perplexity Ranking

Installing Dependencies

In [ ]:
!pip install -q -U einops

Importing Dependencies

In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
from torch import nn
from transformers import AutoTokenizer, AutoModelForCausalLM

import warnings
warnings.simplefilter("ignore") # Ignore warnings

Define the model id

In [ ]:
model_id = 'tiiuae/falcon-7b-instruct'

Load tokenizer from HuggingFace library using the given model id

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token # set pad token as eos token

Load model from HuggingFace library using the given model id

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,  # use float16 for model for lesser memory footprint
    device_map="auto",  # automatically maps the model
    trust_remote_code=True,
    revision="main"  # use main branch of the model
)

Load the training and testing data

In [ ]:
train = pd.read_csv('/kaggle/input/kaggle-llm-science-exam/train.csv')
test = pd.read_csv('/kaggle/input/kaggle-llm-science-exam/test.csv')

Define Perplexity class

In [ ]:
class Perplexity(nn.Module):
    """
    A class used to compute the perplexity of model's predictions
    """
    def __init__(self, reduce: bool = True):
        super().__init__()
        self.loss_fn = nn.CrossEntropyLoss()
        self.reduce = reduce

    def forward(self, logits, labels):
        # Shift the labels and logits by one position to calculate loss
        shift_logits = logits[..., :-1, :].contiguous()
        shift_labels = labels[..., 1:].contiguous()

        # Compute loss for each item in the batch
        perplexity = [self.loss_fn(shift_logits[i], shift_labels[i]) for i in range(labels.shape[0])]
        perplexity = torch.stack(perplexity, dim=0)

        # Compute mean if reduce is set to True
        if self.reduce:
            perplexity = torch.mean(perplexity)

        return perplexity

In [ ]:
# Move Perplexity object to GPU
perplexity_calculator = Perplexity().to("cuda")

Function to calculate precision at k

In [ ]:
def calculate_precision_at_k(r, k):
    """
    Calculates precision at k for ranked results.
    """
    assert k <= len(r)
    assert k != 0
    return sum(int(x) for x in r[:k]) / k

Function to calculate mean average precision at 3

In [ ]:
def calculate_MAP_at_3(predictions, true_items):
    """
    Calculates mean average precision at 3.
    """
    U = len(predictions)
    map_at_3 = 0.0

    for u in range(U):
        user_preds = predictions[u]
        user_true = true_items[u]
        user_results = [1 if item == user_true else 0 for item in user_preds]

        for k in range(min(len(user_preds), 3)):
            map_at_3 += calculate_precision_at_k(user_results, k+1) * user_results[k]

    return map_at_3 / U

In [ ]:
map_scores = []
predictions_list = []

For each row in the training data

In [ ]:
for idx, row in tqdm(train.iterrows(), total=len(train)):
    with torch.no_grad():
        cols = ["A", "B", "C", "D", "E"]
        perplexities = []
        samples = [f"{row['prompt']}</s>{row[col]}" for col in cols]

        # Prepare input for the model
        inputs = tokenizer(
            samples, 
            return_tensors="pt", 
            add_special_tokens=False, 
            padding=True, 
            truncation=True
        ).to("cuda")

        # Get model's output
        output = model(
            input_ids=inputs["input_ids"], 
            attention_mask=inputs["attention_mask"]
        ).logits

        # Prepare labels
        labels = inputs["input_ids"]
        labels.masked_fill_(~inputs["attention_mask"].bool(), -100)

        # Calculate perplexity for each choice
        for j in range(len(cols)):
            perplexity = perplexity_calculator(output[j].unsqueeze(0), labels[j].unsqueeze(0))
            perplexities.append(perplexity.detach().cpu())

        # Delete tensors to free up GPU memory
        del inputs, labels, output, perplexity

    perplexities = np.array(perplexities)

    # Sort predictions according to perplexities
    sorted_predictions = np.array(cols)[np.argsort(perplexities)]
    predictions_list.append([sorted_predictions])

    true_answer = [row.answer]
    map_score = calculate_MAP_at_3([sorted_predictions], true_answer)
    map_scores.append(map_score)

Mean Average Precision for the training data

In [ ]:
mean_map_score = np.mean(map_scores)
print(f"Mean Average Precision (MAP) score is: {mean_map_score}")

Load sample submission csv

In [ ]:
submission = pd.read_csv('sample_submission.csv')

Write predictions to csv file

In [ ]:
submission["prediction"] = [" ".join(p[0][:3]) for p in preds]

In [ ]:
submission.head()

Save the predictions to csv file

In [ ]:
submission.to_csv('submission.csv', index=False)